In [1]:
# Dataframe
import pandas as pd

# Array
import numpy as np

# Datetime
from datetime import datetime

import ast

In [2]:
df_reviews = pd.read_csv('reviews5.csv')
df_meta = pd.read_csv('meta.csv')

/Users/tanyh/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/tanyh/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
def clean_price(df_1):
    df = df_1.copy()
    df = df[~df['price'].str.contains('px', na=False)]
    df = df[~df['price'].str.contains('n', na=False)]
    df = df[~df['price'].str.contains('-', na=False)]
    df = df[~df['price'].str.contains('\n\n\n<script', na=False)]

    #Delete dollar sign and comma from price column and convert to float
    df['price'] = df['price'].str.replace('$', '')
    df['price'] = df['price'].str.replace(',', '')

    df['price'] = df['price'].astype(float)
    return df

df = df_meta.copy()
df = clean_price(df)

In [4]:
def extract_rank(rank_list):
    try:
        l = ast.literal_eval(rank_list)
    except:
        return None
    for r in l:
        if r[-28:] == "(See Top 100 in Electronics)" or r[-25:] == "Electronics (See top 100)":
            return float(r.split()[0][2:].replace(',', ''))

df['rankElectronics'] = df['rank'].apply(extract_rank)
df = df.dropna(subset=['rankElectronics', 'price', 'brand', 'main_cat', 'title']).reset_index()

In [5]:
# Filter Irrelevant Categories from Electronics

main_cat_list = list(df['main_cat'].unique())

exclude_list = ['Books', 
                '<img src="https://images-na.ssl-images-amazon.com/images/G/01/nav2/images/gui/amazon-fashion-store-new._CB520838675_.png" class="nav-categ-image" alt="AMAZON FASHION" />',
                'Grocery',
                'Pet Supplies',
                '<img src="https://m.media-amazon.com/images/G/01/nav2/images/gui/amazon-fashion-store-new._CB520838675_.png" class="nav-categ-image" alt="AMAZON FASHION"/>',
                'Gift Cards',
                '<img src="https://images-na.ssl-images-amazon.com/images/G/01/img16/amazon-launchpad/content-grid/launchpad_lp_nav-logo_v2._CB268684856_.png" class="nav-categ-image" alt="AMAZON LAUNCHPAD"/>',
                '<img src="https://images-na.ssl-images-amazon.com/images/G/01/nav2/images/gui/amazon-fashion-store-new._CB520838675_.png" class="nav-categ-image" alt="AMAZON FASHION"/>',
               ]

final_list = [x for x in main_cat_list if x not in exclude_list]
df = df.loc[df['main_cat'].isin(final_list)]

In [6]:
# To formulate the datetime

import re

def concat_ele(word_list):
    new_list = []
    for w in word_list:
        w = re.sub("[^0-9]", "", w)
        new_list.append(w)
    return new_list

def built_string(word_list):
    e1 = word_list[0]
    e2 = word_list[1]
    e3 = word_list[2]
    new_list = [e3, e1, e2]
    return ''.join(new_list)

In [7]:
df_rev = df_reviews.copy()

# split string into array
df_rev['reviewTime'] = df_rev['reviewTime'].apply(lambda x: x.split())

# remove non digits characters, such as ,
df_rev['reviewTime'] = df_rev['reviewTime'].apply(lambda x: concat_ele(x))

# reorder the string of dates
df_rev['reviewTime'] = df_rev['reviewTime'].apply(lambda x: built_string(x))

# make into datetime format
df_rev['reviewTime'] = pd.to_datetime(df_rev['reviewTime'], format='%Y%m%d')

# keep only reviews from 2010
df_rev = df_rev[df_rev['reviewTime'] > '2010-01-01']

In [8]:
# output cleaned dfs to csv for further analysis

df_rev.to_csv("df_reviews.csv")
df.to_csv("df_meta.csv")